In [37]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import pylab as plt
from firecrown.likelihood.likelihood import load_likelihood
from firecrown.likelihood.likelihood import load_likelihood_from_module_type
from firecrown.parameters import ParamsMap
import numpy as np
import sacc
import pathlib
import pyccl as ccl
from firecrown.modeling_tools import ModelingTools
import firecrown
import sys
import os
from copy import deepcopy
# we need to import a firecrown example likelihood:
# exports the path to the environment variable FIRECROWN_DIR
os.environ["FIRECROWN_DIR"] = os.path.dirname('/Users/loureiro/Work/Github/firecrown/')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
# importing the example likelihood module from the firecrown examples
sys.path.append(os.path.join(os.environ["FIRECROWN_DIR"], "examples", "des_y1_3x2pt"))
# trying to import the example likelihood
import des_y1_3x2pt_PT 

In [39]:
sys.path.append('../src/blinding/')
from smokescreen import Smokescreen

In [66]:
# ccl cosmology
cosmo = ccl.Cosmology(Omega_c=0.27, 
                      Omega_b=0.045, 
                      h=0.67, 
                      sigma8=0.8, 
                      n_s=0.96, 
                      transfer_function='bbks')
sacc_data = sacc.Sacc.load_fits('/Users/loureiro/Work/Github/firecrown/examples/des_y1_3x2pt/des_y1_3x2pt_sacc_data.fits')

# FIXME: not sure there's another way of setting this from the likelihood
syst_dict = {
            "ia_a_1": 1.0,
            "ia_a_2": 0.5,
            "ia_a_d": 0.5,
            "lens0_bias": 2.0,
            "lens0_b_2": 1.0,
            "lens0_b_s": 1.0,
            "lens0_mag_bias": 1.0,
            "src0_delta_z": 0.000,
            "lens0_delta_z": 0.000,}

In [71]:
smkscr = Smokescreen(cosmo, syst_dict, sacc_data, des_y1_3x2pt_PT, {'Omega_c': 0.35}, **{'debug': True})

Using parameters: ['src0_delta_z', 'lens0_bias', 'lens0_delta_z', 'ia_a_1', 'ia_a_2', 'ia_a_d', 'lens0_b_s', 'lens0_b_2', 'lens0_mag_bias']
[DEBUG] Shifts: {'Omega_c': 0.35}
[DEBUG] Blinded Cosmology: <pyccl.cosmology.Cosmology>
	Neff    = 3.044
	Omega_b = 0.045
	Omega_c = 0.35
	h       = 0.67
	n_s     = 0.96
	sigma8  = 0.8
	extra_parameters =
	HASH_ACCURACY_PARAMS = 0xd7a5be1fbac8830


In [72]:
smkscr._create_blinded_cosmo()

<pyccl.cosmology.Cosmology>
	Neff    = 3.044
	Omega_b = 0.045
	Omega_c = 0.35
	h       = 0.67
	n_s     = 0.96
	sigma8  = 0.8
	extra_parameters =
	HASH_ACCURACY_PARAMS = 0xd7a5be1fbac8830